# One run final walkthrough

Do the full run walkthrough on the large data set



Refractor the source code and bring it to individual scripts

Ensure a full run through with one click

In [1]:
## check some parameters
## depending where you launch your notebook, the relative path might not work
## you should start the notebook server from your base path
## when opening the notebook, typically your path will be ../ads_covid-19/notebooks
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: DataScience'

## 1. Update all data

In [2]:
# %load C:\Users\Nitin\ds-covid19\src\data\get_data.py
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chidr("../")

def get_johns_hopkins():
    git_pull = subprocess.Popen("git pull" ,
                                cwd = os.path.dirname('C:/Users/Nitin/ds-covid19/data/raw/COVID-19/'),
                                shell = True,
                                stdout = subprocess.PIPE,
                                stderr = subprocess.PIPE)
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


def get_current_data_germany():
    # 16 states
    #data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    # 400 regions/ Landkreise
    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')
    json_object=json.loads(data.content)
    full_list=[]
    for pos,each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])

        pd_full_list=pd.DataFrame(full_list)
        pd_full_list.to_csv('C:/Users/Nitin/ds-covid19/data/raw/NPGEO/Ger_state_data.csv',sep=':')
        #print('Number of regions rows: '+str(pd_full_list.shape[0]))

if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()


Error : b''
out : b'Already up to date.\n'


## 2. Process Pipeline

In [3]:
# %load C:\Users\Nitin\ds-covid19\src\data\process_combined_data.py
import pandas as pd
import numpy as np
import requests
import subprocess
import os

from datetime import datetime


def store_relational_JH_data():
    data_path='C:/Users/Nitin/ds-covid19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                                   'Province/State':'state'})
    pd_data_base['state']=pd_data_base['state'].fillna('no')
    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country'])\
                                .T                             \
                                .stack(level=[0,1])            \
                                .reset_index()                 \
                                .rename(columns={'level_0':'date',
                                                0:'confirmed'},
                                               )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('C:/Users/Nitin/ds-covid19/data/processed/COVID_relational_confirmed.csv',sep=';')
    #print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    #print(' Latest date is: '+str(max(pd_relational_model.date)))

def store_flat_table_JH_data():
    "process raw JH data into a flat table data structure"
    datapath='C:/Users/Nitin/ds-covid19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    JH_data_raw=pd.read_csv(datapath)
    time_index=JH_data_raw.columns[4:]
    pd_flat_table=pd.DataFrame({'date':time_index})
    country_list=JH_data_raw['Country/Region'].unique()
    for country in country_list:
        pd_flat_table[country]=np.array(JH_data_raw[JH_data_raw['Country/Region']==country].iloc[:,4::].sum(axis=0))
    time_index=[datetime.strptime(each,"%m/%d/%y") for each in pd_flat_table.date]
    pd_flat_table['date']=time_index
    pd_flat_table.to_csv('C:/Users/Nitin/ds-covid19/data/processed/COVID_JH_flat_table_confirmed.csv',sep=';',index=False )
    print('Latest date is'+str(max(pd_flat_table.date)))
    print(' Number of rows stored: '+str(pd_flat_table.shape[0]))

if __name__ == '__main__':

    store_relational_JH_data()
    store_flat_table_JH_data()


Latest date is2020-09-14 00:00:00
 Number of rows stored: 237


## 3. Filter and Doubling Rate Calculation

In [4]:
# %load C:\Users\Nitin\ds-covid19\src\features\build_features.py
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Error in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Error in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('C:/Users/Nitin/ds-covid19/data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('C:/Users/Nitin/ds-covid19/data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())


the test slope is: [2.]
      Unnamed: 0        date state  country  confirmed  confirmed_filtered  \
34360      61856  2020-09-10    no  Germany   258149.0            258018.2   
34361      62122  2020-09-11    no  Germany   259735.0            259374.2   
34362      62388  2020-09-12    no  Germany   260817.0            260732.0   
34363      62654  2020-09-13    no  Germany   261737.0            261946.8   
34364      62920  2020-09-14    no  Germany   263222.0            263161.6   

       confirmed_DR  confirmed_filtered_DR  
34360    160.722431             168.789051  
34361    156.332930             184.661656  
34362    194.577961             191.152480  
34363    260.502498             202.662158  
34364    217.817325             215.629569  


## 4. Visual Board

In [ ]:
# %load C:\Users\Nitin\ds-covid19\src\visualization\visualization_combined.py 
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import dash_bootstrap_components as dbc
import plotly.io as pio

import os
print(os.getcwd())
df_input_large=pd.read_csv('C:/Users/Nitin/ds-covid19/data/processed/COVID_final_set.csv',sep=';')
df_SIR_large=pd.read_csv('C:/Users/Nitin/ds-covid19/data/processed/COVID_JH_flat_table_confirmed.csv',sep=';',parse_dates=[0])
df_SIR_large=df_SIR_large.sort_values('date',ascending=True)

fig=go.Figure()
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.title = 'COVID-19 Dashboard based on Applied Data Science'

app.layout = html.Div([

        dbc.Row(dbc.Col(html.H1('COVID-19 Data Dashboard Visualization using Applied Data Science'),
                        width={'size': 8, 'offset': 1},
                        ),
                ),

        dbc.Row(dbc.Col(html.Div('''
                            Goal of the project is to teach data science by applying a cross industry standard process,
                            it covers the full walkthrough of: automated data gathering, data transformations,
                            filtering and machine learning to approximating the doubling time, and
                            (static) deployment of responsive dashboard alongwith SIR simulation.
                            '''),
                        width={'size': 8, 'offset': 1},
                        )
                ),


       dbc.Row(dbc.Col(html.H5('Select a single country for SIR simulation curve'),
                        width={'size': 5, 'offset': 1},
                        ),
                ),

        dbc.Row(
        [
                  dbc.Col(

                        dcc.Dropdown( id='single_select_country',
                             options=[{'label':each,'value':each} for each in df_SIR_large.columns[1:]],
                             value='Germany',
                             multi=False),
                             width={'size': 5, "offset": 1, 'order': 'second'}),
            ],
        ),


        dbc.Row(dbc.Col(html.H5('In order to manipulate the SIR curve, vary the values regarding the measures and press enter:'),
                        width={'size': 5, 'offset': 1},
                        ),
                ),


        dbc.Row(
        [

            #For changing beta ,gamma, t_initial, t_intro_measures,t_hold,t_relax
            dbc.Row(children=[
            html.Br(),
            html.Br(),
            html.Label(["No measures introduced (in days):",
                      dcc.Input(id='t_initial',
                     type='number',
                     value=28,debounce=True)],style={"margin-left": "30px"}),
            html.Label(["Measures introduced over (in days):",
                      dcc.Input(id='t_intro_measures',
                     type='number',
                     value=14,debounce=True)],style={"margin-left": "30px"}),
            html.Label(["Introduced measures hold time (in days):",
                      dcc.Input(id='t_hold',
                     type='number',
                     value=21,debounce=True)],style={"margin-left": "30px"}),
            html.Br(),
            html.Br(),
            html.Label(["Introduced measures relaxed (in days):",
                      dcc.Input(id='t_relax',
                     type='number',
                     value=21,debounce=True)],style={"margin-left": "30px"}),
            html.Label(["Beta max:",
                      dcc.Input(id='beta_max',
                     type='number',
                     value=0.4,debounce=True)],style={"margin-left": "30px"}),
            html.Label(["Beta min:",
                      dcc.Input(id='beta_min',
                     type='number',
                     value=0.11,debounce=True)],style={"margin-left": "30px"}),
            html.Label(["Gamma:",
                      dcc.Input(id='gamma',
                     type='number',
                     value=0.1,debounce=True)],style={"margin-left": "30px"}),
            html.Br(),
            html.Br(),
            ]
            ),


                dbc.Col(dcc.Graph(
                            figure=fig,
                            id='SIR_curve'),
                        width=6, md={'size': 10,  "offset": 1, 'order': 'last'}
                        ),
            ]
        ),

         dbc.Row(dbc.Col(html.H5('Multi - Select Country for Visualization'),
                        width={'size': 5, 'offset': 1},
                        ),
                ),


        dbc.Row(
            [
                dbc.Col(
                       dcc.Dropdown(
                      id='country_drop_down',
                      options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
                      value=['US', 'Germany','Italy'], # which are pre-selected
                      multi=True),
                      width={'size': 5, "offset": 1, 'order': 'first'}
                     ),

                ], no_gutters=True
        ),

      dbc.Row(dbc.Col(html.H5('Select Timeline of confirmed COVID-19 cases or the approximated doubling time'),
                        width={'size': 5, 'offset': 1},
                        ),
                ),


        dbc.Row(
            [
                dbc.Col(


                dcc.Dropdown(
                id='doubling_time',
                options=[
                    {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
                    {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
                    {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
                    {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
                ],
                value='confirmed',
                multi=False),
                width={'size': 3, "offset": 1, 'order': 'first'}
                        ),

             ],
        ),

        dbc.Row(
            [
                dbc.Col(dcc.Graph(
                            id='main_window_slope'
                            ),
                        width=6, md={'size': 8,  "offset": 1, 'order': 'first'}
                        ),

                ],
            ),
 ])


@app.callback(
    Output('SIR_curve', 'figure'),
    [Input('single_select_country', 'value'),
    Input('t_initial','value'),
    Input('t_intro_measures','value'),
    Input('t_hold','value'),
    Input('t_relax','value'),
    Input('beta_max','value'),
    Input('beta_min','value'),
    Input('gamma','value')])


def SIR_figure(country,initial_time,intro_measures,hold_time,relax_time,max_beta,min_beta,gamma_max):
    ydata=df_SIR_large[country][df_SIR_large[country]>=30]
    xdata=np.arange(len(ydata))
    N0=5000000
    I0=30
    S0=N0-I0
    R0=0
    gamma=gamma_max
    SIR=np.array([S0,I0,R0])

    t_initial=initial_time
    t_intro_measures=intro_measures
    t_hold=hold_time
    t_relax=relax_time
    beta_max=max_beta
    beta_min=min_beta
    propagation_rates=pd.DataFrame(columns={'susceptible':S0,'infected':I0,'recovered':R0})
    pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                       np.linspace(beta_max,beta_min,t_intro_measures),
                       np.array(t_hold*[beta_min]),
                       np.linspace(beta_min,beta_max,t_relax),
                       ))

    def SIR_model(SIR,beta,gamma):
        'SIR model for simulating spread'
        'S: Susceptible population'
        'I: Infected popuation'
        'R: Recovered population'
        'S+I+R=N (remains constant)'
        'dS+dI+dR=0 model has to satisfy this condition at all time'
        S,I,R=SIR
        dS_dt=-beta*S*I/N0
        dI_dt=beta*S*I/N0-gamma*I
        dR_dt=gamma*I
        return ([dS_dt,dI_dt,dR_dt])

    for each_beta in pd_beta:
        new_delta_vec=SIR_model(SIR,each_beta,gamma)
        SIR=SIR+new_delta_vec
        propagation_rates=propagation_rates.append({'susceptible':SIR[0],'infected':SIR[1],'recovered':SIR[2]},ignore_index=True)

    fig=go.Figure()
    fig.add_trace(go.Bar(x=xdata,
                         y=ydata,
                         marker=dict(color='Lightseagreen'),
                         name='Confirmed Cases'
                        ))

    fig.add_trace(go.Scatter(x=xdata,
                            y=propagation_rates.infected,
                            mode='lines',
                            marker=dict(color='DarkRed'),
                            name='Simulated curve'))

    fig.update_layout(shapes=[
                            dict(type='rect',xref='x',yref='paper',x0=0,y0=0,x1=t_initial,y1=1,fillcolor="MediumPurple",opacity=0.4,layer="below",line_width=0,),
                            dict(type='rect',xref='x',yref='paper',x0=t_initial,y0=0,x1=t_initial+t_intro_measures,y1=1,fillcolor="MediumPurple",opacity=0.5,layer="below",line_width=0,),
                            dict(type='rect',xref='x',yref='paper',x0=t_initial+t_intro_measures,y0=0,x1=t_initial+t_intro_measures+t_hold,y1=1,fillcolor="MediumPurple",opacity=0.6,layer='below',line_width=0,),
                            dict(type='rect',xref='x',yref='paper',x0=t_initial+t_intro_measures+t_hold,y0=0,x1=t_initial+t_intro_measures+t_hold+t_relax,y1=1,fillcolor='MediumPurple',opacity=0.7,layer='below',line_width=0,)
                            ],
                    title='SIR Simulation Model for COVID19',
                    title_x=0.5,
                    xaxis=dict(title='Time (in days)',
                               titlefont_size=16),
                    yaxis=dict(title='Confirmed cases based on Johns Hopkins Data, log scale ',
                               type='log',
                                titlefont_size=16,
                              ),
                    width=1280,
                    height=600,
                     )
    return fig

@app.callback(
     Output('main_window_slope', 'figure'),
     [Input('country_drop_down', 'value'),
     Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


     if 'doubling_rate' in show_doubling:
         my_yaxis={'type':"log",
                'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
               }
     else:
         my_yaxis={'type':"log",
                   'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
               }


     traces = []
     for each in country_list:

         df_plot=df_input_large[df_input_large['country']==each]

         if show_doubling=='doubling_rate_filtered':
             df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
         else:
             df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
        #print(show_doubling)


         traces.append(dict(x=df_plot.date,
                                 y=df_plot[show_doubling],
                                 mode='markers+lines',
                                 opacity=0.9,
                                 name=each
                         )
                 )

     return {
             'data': traces,
             'layout': dict (
                 height=720,

                 xaxis={'title':'Timeline',
                         'tickangle':-45,
                         'nticks':20,
                         'tickfont':dict(size=14,color="#7f7f7f"),
                       },

                 yaxis=my_yaxis
         )
     }
if __name__ == '__main__':
     app.run_server(debug=True,use_reloader=False)


D:\CVT\SS20\DataScience
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
